# keras自定义layer

可以通过继承tf.keras.layers.Layer自定义layer,需要重写一下方法：
1. build：创建weight
2. call：定义计算过程
3. compute_output_shape：计算输出的shape
4. get_config/from_config：可选的，定义序列化过程

In [3]:
import tensorflow as tf
from tensorflow import keras as K
import numpy as np

## 自定义matmul操作

In [5]:
class MyLayer(K.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        
        self.kernel = self.add_weight(name = 'kernel', shape = shape, 
                                      initializer = 'uniform',
                                      trainable = True)
        # 记得调用负类的build方法
        super(MyLayer, self).build(input_shape)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)
    
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

## 创建模型

In [7]:
model = K.Sequential([
    MyLayer(10),
    K.layers.Activation('softmax')
])

In [8]:
model.compile(optimizer = tf.train.RMSPropOptimizer(0.01),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

In [10]:
model.fit(data, labels, batch_size = 32, epochs = 5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
1000/1000 [==============================] - 0s 103us/sample - loss: 11.5457 - acc: 0.1100
Epoch 2/5
1000/1000 [==============================] - 0s 31us/sample - loss: 11.5467 - acc: 0.0960
Epoch 3/5
1000/1000 [==============================] - 0s 28us/sample - loss: 11.5561 - acc: 0.1190
Epoch 4/5
1000/1000 [==============================] - 0s 28us/sample - loss: 11.5591 - acc: 0.0890
Epoch 5/5
1000/1000 [==============================] - 0s 30us/sample - loss: 11.5514 - acc: 0.1270
